In [ ]:
"""
Purpose: To check on the status of the Allen split suggestions pipeline
as they run through

- Downloading
- Decimation
- Soma Finding
- Decomposition
- Suggestion Splits

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [3]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

INFO - 2021-03-19 17:16:51,769 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-19 17:16:51,771 - settings - Setting database.user to celiib
INFO - 2021-03-19 17:16:51,773 - settings - Setting database.password to newceliipass
INFO - 2021-03-19 17:16:51,782 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-19 17:16:51,783 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-19 17:16:51,796 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-19 17:16:52,182 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-19 17:16:52,186 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-19 17:16:52,187 - settings - Setting database.user to celiib
INFO - 2021-03-19 17:16:52,188 - settings - Setting database.password to newceliipass
INFO - 2021-03-19 17:16:52,189 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-19 17:16:52,190 - settings - Setting database.user to celiib
INFO - 2021-03-19 17:16:52,191 - settings - Setting database.password to newceliipass
INFO - 2021-03-19 17:16:52,193 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-19 17:16:52,609 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-19 17:16:52,808 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-19 17:16:52,809 - settings - Setting database.user to celiib
INFO - 2021-03-19 17:16:52,810 - settings - Setting database.password to newceliipass
INFO - 2021-03-19 17:16:52,814 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-19 17:16:53,284 - settings - Setting enable_python_native_blobs to True


# Investigating why some neurons did not get split suggestions

In [5]:
allen_table = (minnie.AllenProofreading() & dict(month=3,day=18,year=2021))
allen_table

segment_id segment id for those to be decimated,month the month that the cell was added,day the day that the cell was added,year the year that the cell was added,n_somas the number of somas associated with the neuron from the Allen Institute
864691135081756919,3,18,2021,3
864691135132762784,3,18,2021,3
864691135337764326,3,18,2021,3
864691135340990917,3,18,2021,3
864691135341029829,3,18,2021,2
864691135345208735,3,18,2021,2
864691135345280671,3,18,2021,3
864691135373423816,3,18,2021,2
864691135382504922,3,18,2021,3
864691135382719450,3,18,2021,3


In [6]:
minnie.BaylorSegmentCentroid() & allen_table  & dict(decimation_ratio=0.25) & "multiplicity=0"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,ver the version number of the materializaiton,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,mesh datajoint adapter to get the somas mesh objects,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,volume the volume in billions (10*9 nm^3) of the convex hull,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)
864691136236789519,0,0.25,0,30.00,None,None,None,0,0,=BLOB=,0,nan,nan,nan,nan,nan,846.4636449813843


In [7]:
nviz.plot_objects(du.fetch_segment_id_mesh(864691136236789519))

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [ ]:
key_source = minnie.Decomposition() & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source

In [ ]:
key_source & "n_multi_soma_touching_limbs = 0 AND n_same_soma_multi_touching_limbs=0 "

In [ ]:
(minnie.Decomposition() & dict(segment_id=864691136008729518))

In [ ]:
neuron_obj = (minnie.Decomposition() & dict(segment_id=864691136008729518)).fetch1("decomposition")
neuron_obj

In [ ]:
import datajoint_utils as du
neuron_obj = du.fetch_neuron_obj_manual(segment_id=864691136008729518)

In [ ]:
import neuron_utils as nru

In [ ]:
segment_id = 864691135782479155
dec_mesh = du.fetch_segment_id_mesh(segment_id)
neuron_obj = nru.decompress_neuron("/mnt/dj-stor01/platinum/minnie65/02/decomposition/864691135782479155_0_25.pbz2",
                     original_mesh=dec_mesh)

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

In [ ]:
neuron_obj.n_error_limbs

In [ ]:
nviz.visualize_neuron_lite(neuron_obj)

In [ ]:
segment_id_with_no_errors = [
    864691135345291423, #had just soma on the very tip
    864691135373423816, #removed in glia
    864691135753669837,
    ,
]

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'") & "status='error'"
(curr_table).delete()# & "status='error'")

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
seg_ids_to_delete = [k["segment_id"] for k in curr_table.fetch("key")]
seg_ids_to_delete

In [ ]:
minnie.Decomposition()

# All the cells that need to get suggestions

In [ ]:
key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source

In [ ]:
no_splits = (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)) - key_source.proj()
no_splits

In [ ]:
minnie.DecompositionSplit() & key_source.proj()

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
(curr_table)#.delete()# & "status='error'")

# Getting the split suggesstions

In [ ]:
import datajoint as dj
dj.U("segment_id") & minnie.DecompositionSplit() & key_source.proj()

In [ ]:
du.create_suggested_splits_neuroglancer_spreadsheet?

In [ ]:
allen_segment_ids = (minnie.AllenProofreading() & dict(day=3,month=3,year=2021)).fetch("segment_id")
du.create_suggested_splits_neuroglancer_spreadsheet(segment_ids=allen_segment_ids,output_type="server",
                                                   print_links=True,
                                                   output_filename="allen_spreadsheet_3_10.csv")